Example using python and a SPARQL query that extract "In Riksdagen kallad" name form stored in the Wikidata property P2561 to a pandas [Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
* This [notebook](https://github.com/salgo60/Wikidata_riksdagen-corpus/blob/main/Notebook/.ipynb_checkpoints/WikidatatoPandas-checkpoint.ipynb)
  * GITHUB issue [issues/103](https://github.com/welfare-state-analytics/riksdagen-corpus/issues/103#issuecomment-1012289603)
* example swedish PM [Hjalmar Branting](https://en.wikipedia.org/wiki/Hjalmar_Branting) and his Wikidata object [Q53620#P2561](https://www.wikidata.org/wiki/Q53620#P2561) how its stored in Wikidata
  * related information
    * [list generated](https://sv.wikipedia.org/wiki/Anv%C3%A4ndare:Salgo60/Iriksdagenkallad) of what we have in Wikidata with "In Riksdagen kallad"
       * same list but with [pictures](https://sv.wikipedia.org/wiki/Anv%C3%A4ndare:Salgo60/Iriksdagenkalladbild)
       * [list](https://sv.wikipedia.org/wiki/Anv%C3%A4ndare:Salgo60/Iriksdagenkalladsamma) were the same namestring is used - Wikidata has today about 10% done
    * template [Iriksdagenkallad](https://sv.wikipedia.org/wiki/Mall:Iriksdagenkallad) created in sv:WIkipedia to present this Wikidata text string in Swedish Wikipedia article
    * sv:Wikipedia article explaining "[In Riksdagen kallad](https://sv.wikipedia.org/wiki/I_riksdagen_kallad)"

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-01-14 02:50:27.263419


In [2]:
# function that returns pandas Dataframe
import sys,json
import pandas as pd 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [10]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#title: Images people with nameform "I Riksdagen kallad"
SELECT DISTINCT ?nameUsedinSwedishPM ?nr ?item ?itemLabel ?itemAltLabel ?img ?svWikipedia WHERE {
  ?item p:P2561 ?nameSwedishPMp.
  OPTIONAL { ?item wdt:P18 ?img. }
  {
    {
      ?nameSwedishPMp ps:P2561 ?nameUsedinSwedishPM;
        pq:P3831 wd:Q110382440.
      OPTIONAL { ?nameSwedishPMp pq:P1545 ?nr. }
    }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
  OPTIONAL {
    ?svWikipedia schema:about ?item;
      schema:inLanguage "sv";
      schema:isPartOf <https://sv.wikipedia.org/>.
  }
}
ORDER BY (?nameUsedinSwedishPM) (?nr)"""




WDRiksdagentot = get_sparql_dataframe(endpoint_url, query)

In [11]:
WDRiksdagentot.shape

(553, 7)

In [12]:
WDRiksdagentot

,nameUsedinSwedishPM,nr,item,itemLabel,itemAltLabel,img,svWikipedia
0,Abrahamsson i Jönköping,1,http://www.wikidata.org/entity/Q5627413,Erik Edström,"Erik Rudolf Abrahamsson, Abrahamsson i Jönköpi...",http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Erik_Abrahamsson...
1,Adelsköld i Nolhaga o Stockholm,1,http://www.wikidata.org/entity/Q384847,Claes Adolf Adelsköld,Adelsköld i Nolhaga o Stockholm,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Claes_Adelsk%C3%...
2,Adlersparre i Arvika,2,http://www.wikidata.org/entity/Q5542896,Rudolf Adlersparre,"Adlersparre i Arvika, Adlersparre i Gustafsvik",http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Rudolf_Adlersparre
3,Adlersparre i Gustafsvik,1,http://www.wikidata.org/entity/Q5542896,Rudolf Adlersparre,"Adlersparre i Arvika, Adlersparre i Gustafsvik",http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Rudolf_Adlersparre
4,Adolfsson i Göteborg,2,http://www.wikidata.org/entity/Q5410165,Gunnar Adolfsson,"Adolfsson i Göteborg, Adolfsson i Malmö",http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Gunnar_Adolfsson
...,...,...,...,...,...,...,...
548,Åkerman i Strömstad,None,http://www.wikidata.org/entity/Q6255333,Assar Åkerman,Åkerman i Strömstad,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Assar_%C3%85kerman
549,Öhman i Stockholm,1,http://www.wikidata.org/entity/Q6256712,Gunnar Öhman,Öhman i Stockholm,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Gunnar_%C3%96hman
550,Öhvall i Bergnäset,None,http://www.wikidata.org/entity/Q6256938,Bertil Öhvall,"Öhvall i Bergnäset, Oscar Bertil Öhvall",http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Bertil_%C3%96hvall
551,Östling i Valbo,1,http://www.wikidata.org/entity/Q6258086,Olof Östling,Östling i Valbo,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Olof_%C3%96stling


In [13]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-01-14 02:53:13.098498
Time elapsed (hh:mm:ss.ms) 0:02:45.837000
